In [ ]:
import numpy as np
import cv2
import face_recognition
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import model_from_json

In [ ]:
vid = cv2.VideoCapture(0)   # 0 for webcam

#load the model and load the weights
face_exp_model = model_from_json(open("facial_expression_model_structure.json","r",encoding= "utf-8").read())
face_exp_model.load_weights('facial_expression_model_weights.h5')
emotions_label = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

while True:
    
    success, img = vid.read()
    num_faces = face_recognition.face_locations(img, number_of_times_to_upsample=1, model = "hog")
    
    for index, location in enumerate(num_faces):
        top, right, bottom, left = location
        face_image = img[top:bottom,left:right]
        cv2.rectangle(img, (left,top), (right,bottom), (0,0,200), 2)
        
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY) 
        face_image = cv2.resize(face_image, (48, 48))
        face_image = image.img_to_array(face_image)
        face_image = np.expand_dims(face_image, axis = 0)
        face_image = face_image/255  # normalize all pixels in scale of [0, 1]
        
        prediction = face_exp_model.predict(face_image) 
        max_index = np.argmax(prediction[0])
        emotion_label = emotions_label[max_index]

        #display the name as text in the image
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(img, emotion_label, (left,bottom), font, 0.5, (255,255,255),1)
        
    cv2.imshow("Faces", img)             # should be outside for loop else won't detect non face images.
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
vid.release()
cv2.destroyAllWindows()  

-----------